# DEMO: HF calculations on small organic molecules by PySCF

* We need to install a few things before we get started
  * [PySCF](https://pyscf.org/) (for the quantum chemistry)
  * [py3DMol](https://3dmol.csb.pitt.edu/) for visualizing the molecule
  * [plotly](https://plotly.com/python/) and kaleido for plotting

In [ ]:
%pip install -q pyscf py3DMol plotly kaleido

In [ ]:
from pyscf import gto   # Used to define a molecule
from pyscf import scf   # Used to perform HF calculations
from pyscf import mp    # Used to perform Møller–Plesset PT calculations
from pyscf import cc    # Used to perfrom Coupled Cluster calculations
from pyscf import mcscf # Used to perform multireference calculations

### Build and Visualize Molecules

In [ ]:
mol_xyz = """H      1.2194     -0.1652      2.1600
  C      0.6825     -0.0924      1.2087
  C     -0.7075     -0.0352      1.1973
  H     -1.2644     -0.0630      2.1393
  C     -1.3898      0.0572     -0.0114
  H     -2.4836      0.1021     -0.0204
  C     -0.6824      0.0925     -1.2088
  H     -1.2194      0.1652     -2.1599
  C      0.7075      0.0352     -1.1973
  H      1.2641      0.0628     -2.1395
  C      1.3899     -0.0572      0.0114
  H      2.4836     -0.1022      0.0205"""

mol = gto.M(atom=mol_xyz, basis="sto3g", verbose=3)

In [ ]:
import py3Dmol

xyz_view = py3Dmol.view(width=400,height=400)
xyz_view.addModel(mol.tostring(format="xyz"),'xyz')
xyz_view.setStyle({'stick':{}, "sphere":{"radius":0.4}})
xyz_view.setBackgroundColor('0xeeeeee')
xyz_view.show()

#
# Use your mouse to interact with the molecule!
#

### Run calculations

In [ ]:
# Run HF and save the results
mf = scf.RHF(mol).run()

# In Jupyter notebooks you can hover over methods to see docstrings or you can print the docstrings out!
#print(mf.__doc__)

### Visualize Results

In [ ]:
import plotly.express as px

# Plot the MO Occupations
fig = px.line(y=mf.mo_occ, markers=True, title="Molecular Orbital (MO) Occupations")
fig.update_layout(xaxis_title="Orbital Index (0-Based)", yaxis_title="MO Occupation")
fig.show()

In [ ]:
# Plot the MO Energies (i.e. eigenvalues of the Fock matrix)
fig = px.line(y=mf.mo_energy, markers=True, title="Molecular Orbital (MO) Energies")
fig.update_layout(xaxis_title="Orbital Index (0-Based)", yaxis_title="MO Energies")
fig.show()

In [ ]:
from pyscf.tools import cubegen

cubegen.orbital(mol, 'mo.cube', mf.mo_coeff[:,18]); # 42 electrons so 21 occupied orbitals

cube_view = py3Dmol.view(width=400,height=400)
cube_view.addVolumetricData(open("mo.cube").read(), "cube", {'isoval': -0.03, 'color': "red", 'opacity': 0.85})
cube_view.addVolumetricData(open("mo.cube").read(), "cube", {'isoval': 0.03, 'color': "blue", 'opacity': 0.85})
cube_view.addModel(mol.tostring(format="xyz"),'xyz')
cube_view.setStyle({'stick':{}, "sphere":{"radius":0.4}})
cube_view.setBackgroundColor('0xeeeeee')
cube_view.show()

#
# Use your mouse to interact with the molecule!
#

### Short Survey of more acurate methods

In [ ]:
from pyscf import gto, scf, dft, mp, cc, fci
import py3Dmol
import plotly.express as px

In [ ]:
# Experimental geometry of gas-phase water
# Ref: https://cccbdb.nist.gov/expgeom2x.asp
mol_xyz = """O        0.0000   0.0000   0.1173
             H        0.0000   0.7572	 -0.4692
             H        0.0000  -0.7572	 -0.4692"""
mol = gto.M(
    atom=mol_xyz, 
    basis="6-31g", 
    verbose=4,
    charge=0,      # 0 by default
    spin=0,        # 0 by default, defined as (n_up - n_down)
    symmetry=True, # False by default
)


In [ ]:
xyz_view = py3Dmol.view(width=400,height=400)
xyz_view.addModel(mol.tostring(format="xyz"),'xyz')
xyz_view.setStyle({'stick':{}, "sphere":{"radius":0.4}})
xyz_view.setBackgroundColor('0xeeeeee')
xyz_view.show()

#### [Hartree-Fock](https://en.wikipedia.org/wiki/Hartree%E2%80%93Fock_method)


* Hatree-Fock (HF) is the starting point of the most of quantum chemistry
* We variationally optimize the orbitals for a single [Slater determinint](https://en.wikipedia.org/wiki/Slater_determinant)
* Working in the basis of atom-centered basis function we solve the [Roothaan-Hall](https://en.wikipedia.org/wiki/Roothaan_equations) equations

<!-- $\textbf{FC} = \textbf{SC} \epsilon$

* $\textbf{F}$ is the [Fock matrix]()
* $\textbf{C}$ is the molecular orbital coefficient matrix
* $\textbf{S}$ is the atomic orbital overlap matrix
* $\epsilon$ is the vector of molecular orbital energies -->

See the PySCF [user guide](https://pyscf.org/user/scf.html) and [examples](https://github.com/pyscf/pyscf/tree/master/examples/scf) for more info.



mymf = scf.RHF(mol).run()

#### [Density Functional Theory](https://en.wikipedia.org/wiki/Density_functional_theory)

* In Density Functional Theory (DFT), the electron density of a reference noninteracting system is used to represent the density of the true interacting system.
* The formulation resembles HF with a different effective Fock potential.
* This effective potential depends on the density functional approximation which is chosen by the user.
* PySCF gives users the access to a large number of functionals through the [libxc](https://tddft.org/programs/libxc/) and [xcfun](https://github.com/dftlibs/xcfun) libraries.

See the PySCF [user guide](https://pyscf.org/user/dft.html) and [examples](https://github.com/pyscf/pyscf/tree/master/examples/dft) for more info.

#### [Møller–Plesset perturbation theory](https://en.wikipedia.org/wiki/M%C3%B8ller%E2%80%93Plesset_perturbation_theory)

* Perturbative corrections to the Hartree-Fock approximation.

See the PySCF [user guide](https://pyscf.org/user/mp.html) and [examples](https://github.com/pyscf/pyscf/tree/master/examples/mp) for more info.

#### [Coupled Cluster](https://en.wikipedia.org/wiki/Coupled_cluster)

* Perturbative method that improves on the Hartree-Fock approximation.
* Coupled Cluster Singles and Doubles (CCSD) includes single and double excitation on top of the HF wave function.
* Accuracy can be improved by including triples perturbatively (CCSD(T)).
* Non-variational, but size extensive description of ground states. For excited states, see [EOM-CCSD]().

See the PySCF [user guide](https://pyscf.org/user/cc.html) and [examples](https://github.com/pyscf/pyscf/tree/master/examples/cc) for more info.

In [ ]:
mycc = cc.CCSD(mymf).run()

In [ ]:
e_ccsd_t = mycc.ccsd_t()

#### [Full Configuration Interaction](https://en.wikipedia.org/wiki/Full_configuration_interaction)

* Full configuration interaction (FCI) is exact for a given choice of basis set.
* Cost grows exponentially with the size of the system.
* Also known as exact diagonalization.

See the [PySCF examples](https://github.com/pyscf/pyscf/tree/master/examples/fci) for more details.

In [ ]:
myfci = fci.FCI(mymf)
myfci.kernel();

### Analysis

Important data is saved to the PySCF method objects, making it easy to analyze and visualize!

In [ ]:
# Collect data
methods = ["HF", "MP2", "CCSD", "CCSD(T)", "FCI"]
energies = [mymf.e_tot, mymp2.e_tot, mycc.e_tot, mycc.e_tot + e_ccsd_t, myfci.e_tot]

# Plotting
fig = px.line(x=methods, y=energies, title="Comparison of QC methods", markers=True)
fig.update_layout(xaxis_title="Method", yaxis_title="Energy (Ha)")
fig.update_traces(marker_size=12)
fig.show() # It's interactive!